# 4 Generating the Bank for the MicroTasks

To generate the bank for the microtasks I will use an API for an LLM.
The elemens that I will need are:

1) Question
2) Braoad options (all 6 letters)
3) Shorter options (only 3)
4) The "Reading material"


Basically I will create the perfect prompt that will use the columns of the df_courses to generate the microstasks. 
